# <center> <b> <h1> Gowalla, Stalker pattern recognition </h1></b></center>

<h4 id="challenge">Challenge</h4>

<p>Use the dataset here: <a href="https://snap.stanford.edu/data/loc-gowalla.html">https://snap.stanford.edu/data/loc-gowalla.html</a></p>

<ul>
  <li>Assume a “stalker” is someone who, in this dataset, visits some of the same locations as another person, after the other person goes to that location.</li>
  <li>A “stalker score” for a pair of people, A &amp; B, is the number of locations for which A has visited a location followed by B visiting that same location in the future.</li>
  <li>Any given location should be counted once in the score, so a stalker score can never be higher than the number of unique locations that A and B have in common.</li>
</ul>

<p>Use the datasets from the web page above to answer the following questions:</p>

<ol>
  <li>Which friend pair has the highest “stalker score”?</li>
  <li>Which non-friend pair has the highest “stalker score”?</li>
</ol>

<p>You can use any tools you want to solve this puzzle, except asking for help from other people. Please feel free to email at any time for any clarifications.</p>

<p>Please give the winning user id pairs and “stalker score” for each question, and please explain your solution methods, including any source code if you wrote any.</p>

<p>There are many libraries available that can make this challenge easier, so we are offering bonus points for:</p>
<ul>
  <li>Using as little RAM as possible</li>
  <li>Exection speed</li>
  <li>Pure python solution using base modules</li>
</ul>